In [1]:
import sys
import os
sys.path.append(r"D:\CursorProj\Chem-Ontology-Constructor")
os.environ["PROJECT_ROOT"] = "D:\\CursorProj\\Chem-Ontology-Constructor\\"

In [2]:
from autonogy_constructor.assertions import chemonto_with_entities_assertions, chemonto_with_elements_assertions, chemonto_with_data_properties_assertions, chemonto_with_object_properties_assertions
from autonogy_constructor.ontology_merge import merge_ontology
from src.ontology.preprocess import create_metadata_properties


d:\AnacondaEnPs\envs\OntologyConstruction\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import dspy

lm = dspy.LM('openai/gpt-4o', temperature=0)
dspy.configure(lm=lm)
teacher = dspy.LM("openai/gpt-4o",temperature=1)

In [4]:
def is_doi(line):
    """检查一行文本是否是DOI格式"""
    line = line.strip()
    if line.startswith('https://doi.org/') or line.startswith('10.') or line.startswith('doi.org/') :
        # 检查是否包含多个/符号
        if line.count('/') >= 2:
            return True
    return False

def read_and_split_text(file_path, debug=True):
    with open(file_path, 'r', encoding='utf-8') as f:
        # 标准化所有换行符
        content = f.read().replace('\r\n', '\n')
    
    # 移除每行末尾的空白字符
    lines = [line.rstrip() for line in content.split('\n')]
    
    # 检查第一行是否是DOI
    doi = None
    start_idx = 0
    if lines and is_doi(lines[0]):
        doi = lines[0]
        start_idx = 1
    
    blocks = []
    current_block = []
    block_count = 0
    
    for i, line in enumerate(lines[start_idx:], start=start_idx):
        if debug:
            print(f"Line {i}: '{line}' (Length: {len(line)})")
        
        if not line:  # 遇到空行就分块
            if current_block:
                blocks.append('\n'.join(current_block))
                block_count += 1
                if debug:
                    print(f"找到第 {block_count} 个分块点，在第 {i} 行")
            current_block = []
        else:
            current_block.append(line)
    
    # 添加最后一个块
    if current_block:
        blocks.append('\n'.join(current_block))
        block_count += 1
    
    if debug:
        print(f"\n总共找到 {block_count} 个块")
        lens = [len(block) for block in blocks]
        print(f"每个块的字符长度: {lens}")
        print(f"平均长度: {sum(lens) / len(lens)}")
        if doi:
            print(f"提取到DOI: {doi}")
    
    return blocks, doi

In [5]:

data, doi = read_and_split_text(r'D:\CursorProj\Chem-Ontology-Constructor\tests\Supramolecular\data\supramolecule\accchemres-2023-calix-4-pyrroles review.txt')


Line 1: 'Aryl-Extended and Super Aryl-Extended Calix[4]pyrroles: Design,' (Length: 63)
Line 2: 'Synthesis, and Applications' (Length: 27)
Line 3: 'CONSPECTUS: Proteins exhibit high-binding affinity and' (Length: 54)
Line 4: 'selectivity, as well as remarkable catalytic performance. Their' (Length: 63)
Line 5: 'binding pockets are hydrophobic but also contain polar and' (Length: 58)
Line 6: 'charged groups to contribute to the binding of polar organic' (Length: 60)
Line 7: 'molecules in aqueous solution. In the past decades, the synthesis of' (Length: 68)
Line 8: 'biomimetic receptors featuring sizable aromatic cavities equipped' (Length: 65)
Line 9: 'with converging polar groups has received considerable attention.' (Length: 65)
Line 10: '“Temple” cages, naphthotubes, and aryl-extended calix[4]pyrroles' (Length: 64)
Line 11: 'are privileged examples of synthetic scaffolds displaying function-' (Length: 67)
Line 12: 'alized hydrophobic cavities capable of binding polar substrates. In' (

In [6]:
data = [{"content": text, "source": doi} for text in data]
data

[{'content': 'Aryl-Extended and Super Aryl-Extended Calix[4]pyrroles: Design,\nSynthesis, and Applications\nCONSPECTUS: Proteins exhibit high-binding affinity and\nselectivity, as well as remarkable catalytic performance. Their\nbinding pockets are hydrophobic but also contain polar and\ncharged groups to contribute to the binding of polar organic\nmolecules in aqueous solution. In the past decades, the synthesis of\nbiomimetic receptors featuring sizable aromatic cavities equipped\nwith converging polar groups has received considerable attention.\n“Temple” cages, naphthotubes, and aryl-extended calix[4]pyrroles\nare privileged examples of synthetic scaffolds displaying function-\nalized hydrophobic cavities capable of binding polar substrates. In\nparticular, calix[4]pyrroles are macrocycles containing four pyrrole\nrings connected through their pyrrolic 2- and 5-positions by tetra-\nsubstituted sp3 carbon atoms (meso-substituents). In 1996, Sessler\nintroduced the meso-octamethyl cal

In [7]:
for index, chunk in enumerate(data):
# index = 4
# chunk = data[index]
    print(f"第{index+1}个片段")
    test_context = chunk["content"]
    source = chunk["source"]
    print(f"前30个字符: {test_context[:30]}")
    temp = chemonto_with_entities_assertions(test_context)
    elements = chemonto_with_elements_assertions(test_context, temp)
    data_properties =   chemonto_with_data_properties_assertions(test_context, temp)
    object_properties = chemonto_with_object_properties_assertions(test_context, temp)
    create_metadata_properties()
    merge_ontology(
        temp.ontology_entities, 
        elements.ontology_elements, 
        data_properties.ontology_data_properties, 
        object_properties.ontology_object_properties,
        source
    )

第1个片段
前30个字符: Aryl-Extended and Super Aryl-E
entities
Entities:
  - Name: aryl_extended_calix(4)pyrrole
    Information: Aryl-extended calix[4]pyrroles are macrocycles containing four pyrrole rings connected through their pyrrolic 2- and 5-positions by tetra-substituted sp3 carbon atoms, with aryl substituents in the meso-positions.
  - Name: super_aryl_extended_calix(4)pyrrole
    Information: Super aryl-extended calix[4]pyrroles are derivatives of aryl-extended calix[4]pyrroles with enhanced structural features and binding capabilities.
  - Name: calix(4)pyrrole_cavitand
    Information: Calix[4]pyrrole cavitands are derivatives of calix[4]pyrroles designed to have specific binding cavities for molecular recognition.
  - Name: meso_octamethyl_calix(4)pyrrole
    Information: Meso-octamethyl calix[4]pyrrole is a specific type of calix[4]pyrrole known for its outstanding anion binding properties.
  - Name: two_wall_αα_aryl_extended_calix(4)pyrrole
    Information: Two-wall αα-aryl-exte

2024/12/09 15:35:10 INFO dspy.primitives.assertions: SuggestionFailed: The entity names contain square brackets, which is not standard practice in ontology naming conventions. Removing the square brackets would improve the ontology's adherence to naming standards.


entities
Entities:
  - Name: aryl_extended_calix(4)pyrrole
    Information: Aryl-extended calix(4)pyrroles are macrocycles containing four pyrrole rings connected through their pyrrolic 2- and 5-positions by tetra-substituted sp3 carbon atoms, with aryl substituents in the meso-positions.
  - Name: super_aryl_extended_calix(4)pyrrole
    Information: Super aryl-extended calix(4)pyrroles are derivatives of aryl-extended calix(4)pyrroles with enhanced structural features and binding capabilities.
  - Name: calix(4)pyrrole_cavitand
    Information: Calix(4)pyrrole cavitands are derivatives of calix(4)pyrroles designed to have specific binding cavities for molecular recognition.
  - Name: meso_octamethyl_calix(4)pyrrole
    Information: Meso-octamethyl calix(4)pyrrole is a specific type of calix(4)pyrrole known for its outstanding anion binding properties.
  - Name: two_wall_αα_aryl_extended_calix(4)pyrrole
    Information: Two-wall αα-aryl-extended calix(4)pyrroles feature an aromatic cle

2024/12/09 15:35:27 INFO dspy.primitives.assertions: SuggestionFailed: The entity names contain square brackets, which is not ideal for ontology naming conventions. It is recommended to replace square brackets with parentheses or another suitable character to improve the ontology's compatibility with standard naming practices.
2024/12/09 15:35:38 INFO dspy.primitives.assertions: SuggestionFailed: The entity names contain square brackets, which is not ideal for ontology naming conventions. It is recommended to replace square brackets with parentheses or another suitable character to improve the ontology's compatibility with standard naming practices.


entities
Entities:
  - Name: aryl_extended_calix(4)pyrrole
    Information: Aryl-extended calix(4)pyrroles are macrocycles containing four pyrrole rings connected through their pyrrolic 2- and 5-positions by tetra-substituted sp3 carbon atoms, with aryl substituents in the meso-positions.
  - Name: super_aryl_extended_calix(4)pyrrole
    Information: Super aryl-extended calix(4)pyrroles are derivatives of aryl-extended calix(4)pyrroles with enhanced structural features and binding capabilities.
  - Name: calix(4)pyrrole_cavitand
    Information: Calix(4)pyrrole cavitands are derivatives of calix(4)pyrroles designed to have specific binding cavities for molecular recognition.
  - Name: meso_octamethyl_calix(4)pyrrole
    Information: Meso-octamethyl calix(4)pyrrole is a specific type of calix(4)pyrrole known for its outstanding anion binding properties.
  - Name: two_wall_αα_aryl_extended_calix(4)pyrrole
    Information: Two-wall αα-aryl-extended calix(4)pyrroles feature an aromatic cle

...

entities
Entities:
  - Name: supramolecular_chemistry
    Information: The study of the supramolecular chemistry of AE-C(4)Ps and their derivatives.
  - Name: AE-C(4)Ps
    Information: A type of compound involved in the supramolecular chemistry and binding studies of anions, ion-pairs, and neutral polar molecules.
  - Name: C(4)Ps
    Information: Compounds shown to bind anions, ion-pairs, and neutral polar molecules in solution.
  - Name: anion−π_interactions
    Information: Interactions gauged through experimental energy values in the binding studies of halides with AE-C(4)Ps.
  - Name: AE-C(4)P_cavitands
    Information: Derived from the elaboration of the aromatic cavity of 'four-wall' AE-C(4)Ps.
  - Name: SAE-C(4)Ps
    Information: A derivative of AE-C(4)Ps used in various applications including molecular recognition processes in water.
  - Name: monophosphonate_cavitands
    Information: Used for the recognition and sensing of creatinine.
  - Name: molecular_recognition_proces

2024/12/09 16:09:18 INFO dspy.primitives.assertions: SuggestionFailed: The ontology does not include symmetric aspects of the same chemical scope, which could enhance the completeness and alignment of instances in granularity. To improve, consider adding symmetric properties if applicable, such as detailing the reverse interactions or complementary processes.


properties
Object Properties:
  - Name: has_binding_capability
    Instance:
      Domain:
        Entity: AE-C(4)Ps
        Type: single
      Range:
        Entity: anions, ion-pairs, neutral polar molecules
        Type: union
      Restriction: some
    Information: AE-C(4)Ps have the capability to bind anions, ion-pairs, and neutral polar molecules.
  - Name: facilitates_transport
    Instance:
      Domain:
        Entity: AE-C(4)Ps
        Type: single
      Range:
        Entity: anions, amino acids
        Type: union
      Restriction: some
    Information: AE-C(4)Ps facilitate the transmembrane transport of anions and amino acids.
  - Name: enables_self_assembly
    Instance:
      Domain:
        Entity: AE-C(4)Ps, SAE-C(4)Ps
        Type: union
      Range:
        Entity: bis-C(4)Ps, dimeric capsules, unimolecular metallo-cages
        Type: union
      Restriction: some
    Information: AE- and SAE-C(4)Ps enable the self-assembly of bis-C(4)Ps, dimeric capsules, and unim

2024/12/09 16:09:38 INFO dspy.primitives.assertions: SuggestionFailed: The ontology does not fully align symmetric aspects of the same chemical scope in terms of granularity, particularly in the "are_bound_by" property. To improve, ensure that symmetric properties like "are_bound_by" and "has_binding_capability" are aligned in granularity and specificity.
2024/12/09 16:10:01 INFO dspy.primitives.assertions: SuggestionFailed: The ontology does not fully align symmetric aspects of the same chemical scope in terms of granularity, particularly in the "are_bound_by" property. To improve, ensure that symmetric properties like "are_bound_by" and "has_binding_capability" are aligned in granularity and specificity.


properties
Object Properties:
  - Name: has_binding_capability
    Instance:
      Domain:
        Entity: AE-C(4)Ps
        Type: single
      Range:
        Entity: anions, ion-pairs, neutral polar molecules
        Type: union
      Restriction: some
    Information: AE-C(4)Ps have the capability to bind anions, ion-pairs, and neutral polar molecules.
  - Name: facilitates_transport
    Instance:
      Domain:
        Entity: AE-C(4)Ps
        Type: single
      Range:
        Entity: anions, amino acids
        Type: union
      Restriction: some
    Information: AE-C(4)Ps facilitate the transmembrane transport of anions and amino acids.
  - Name: enables_self_assembly
    Instance:
      Domain:
        Entity: AE-C(4)Ps, SAE-C(4)Ps
        Type: union
      Range:
        Entity: bis-C(4)Ps, dimeric capsules, unimolecular metallo-cages
        Type: union
      Restriction: some
    Information: AE- and SAE-C(4)Ps enable the self-assembly of bis-C(4)Ps, dimeric capsules, and unim

# Improvement

1. 数据属性与对象属性混淆
2. 父类关系归属，是组成部分不能归属为父子类关系
3. 类名称不能有/，#
4. 复合实体需要拆分为多个独立的类
5. 

In [8]:
from config.settings import ONTOLOGY_CONFIG
onto = ONTOLOGY_CONFIG["ontology"]
list(onto.disjoint_classes())   


[]

In [1]:
from owlready2 import *
import owlready2

In [18]:
meta = ONTOLOGY_CONFIG["meta"]
meta_annotations = list(onto.annotation_properties())


In [19]:
default_world.full_text_search_properties.extend(meta_annotations)


In [20]:
default_world.full_text_search_properties

[rdf-schema.label, meta.source, meta.information, meta.hierarchy_information]

In [34]:
res = default_world.search(information = FTS("cryptands"), _bm25 = True)
# print(len(res))
# print(type(res[0][0]))

import pandas as pd

# 将搜索结果转换为DataFrame
df = pd.DataFrame(res, columns=['实体', '相关度分数'])
df


,实体,相关度分数
0,classes.pseudospherical_inner_cavity,-5.244072
1,classes.pyridine_nitrogen_atom,-5.056321
2,classes.naphthyridine_nitrogen_atom,-5.056321
3,classes.oligopyrrolic_cryptand_cage,-5.056321
4,classes.diformyl_pyrrole_precursor,-4.718457
5,classes.cryptate,-4.718457
6,classes.Schiff-base_condensation,-4.565910
7,classes.single-crystal_X-ray_structure,-4.565910
8,classes.molecular_design,-4.565910
9,data_properties.application_area,-4.422917


In [36]:
test = df["实体"][4]
test.information


['Chemical precursors used in the formation of oligopyrrolic cryptands.']

# 推理

In [6]:
from config.settings import ONTOLOGY_CONFIG
onto = ONTOLOGY_CONFIG["ontology"]


In [7]:

owlready2.JAVA_EXE = r"C:\Program Files\Java\jdk-23\bin\java.exe"


In [6]:
close_world(onto)

In [7]:
with onto:
    sync_reasoner(infer_property_values = True)

* Owlready2 * Running HermiT...
    C:\Program Files\Java\jdk-23\bin\java.exe -Xmx2000M -cp d:\AnacondaEnPs\envs\OntologyConstruction\Lib\site-packages\owlready2\hermit;d:\AnacondaEnPs\envs\OntologyConstruction\Lib\site-packages\owlready2\hermit\HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:///C:/Users/SYK/AppData/Local/Temp/tmpy00cw0fl -Y
* Owlready2 * HermiT took 5.924745082855225 seconds
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


In [9]:
with onto:
    sync_reasoner_pellet(infer_property_values = True, infer_data_property_values = True)#, debug = 3)


* Owlready2 * Running Pellet...
    C:\Program Files\Java\jdk-23\bin\java.exe -Xmx2000M -cp d:\AnacondaEnPs\envs\OntologyConstruction\Lib\site-packages\owlready2\pellet\antlr-3.2.jar;d:\AnacondaEnPs\envs\OntologyConstruction\Lib\site-packages\owlready2\pellet\antlr-runtime-3.2.jar;d:\AnacondaEnPs\envs\OntologyConstruction\Lib\site-packages\owlready2\pellet\aterm-java-1.6.jar;d:\AnacondaEnPs\envs\OntologyConstruction\Lib\site-packages\owlready2\pellet\commons-codec-1.6.jar;d:\AnacondaEnPs\envs\OntologyConstruction\Lib\site-packages\owlready2\pellet\httpclient-4.2.3.jar;d:\AnacondaEnPs\envs\OntologyConstruction\Lib\site-packages\owlready2\pellet\httpcore-4.2.2.jar;d:\AnacondaEnPs\envs\OntologyConstruction\Lib\site-packages\owlready2\pellet\jcl-over-slf4j-1.6.4.jar;d:\AnacondaEnPs\envs\OntologyConstruction\Lib\site-packages\owlready2\pellet\jena-arq-2.10.0.jar;d:\AnacondaEnPs\envs\OntologyConstruction\Lib\site-packages\owlready2\pellet\jena-core-2.10.0.jar;d:\AnacondaEnPs\envs\OntologyCon

In [10]:
list(default_world.inconsistent_classes())

[]

In [21]:
classes = ONTOLOGY_CONFIG["classes"]
acetonitrile = classes["acetonitrile"]
print(acetonitrile)

classes.acetonitrile


In [22]:
op = ONTOLOGY_CONFIG["object_properties"]
cocap = op["is_coencapsulated_with"]
cocap.information


['Solvent molecules like acetonitrile and water are coencapsulated with guest molecules in the coordination complex.']

In [23]:
acetonitrile.is_coencapsulated_with



[classes.guest_molecule(29),
 classes.guest_molecule(5a),
 classes.guest_molecule(23a)]

In [51]:
from owlready2 import *
from owlready2.class_construct import Restriction

def find_property_restrictions_from_is_a(cls, property_name):
    restrictions = []
    
    for r in cls.is_a:
        # 检查是否是Restriction类型
        if isinstance(r, Restriction):
            # 获取属性名称并比较
            if r.property.name == property_name:
                # 通过 type 属性来判断限制类型
                restrictions.append(r)
    
    return restrictions

def print_restrictions(cls, property_name):
    restrictions = find_property_restrictions_from_is_a(cls, property_name)
    
    if not restrictions:
        print(f"没有找到属性 {property_name} 的限制")
        return
        
    print(f"\n{cls.name} 类在属性 {property_name} 上的限制：")
    values = []
    for r in restrictions:
        if isinstance(r, Restriction):
            # 使用 type 属性来判断限制类型
            if r.type == ONLY:
                restriction_type = "ONLY"
                print(f"类型: {restriction_type}")
                print(f"值: {r.value}")
                print("---")
                values.append(r.value)
            elif r.type == SOME:
                restriction_type = "SOME"
                print(f"类型: {restriction_type}")
                print(f"值: {r.value}")
                print("---")
                values.append(r.value)
    return tuple(values)
# 使用示例
my_class = classes["acetonitrile"]
property_name = "is_coencapsulated_with"
values = print_restrictions(my_class, property_name)
infos = []
for value in values + (my_class,):
    if isinstance(value, owlready2.entity.ThingClass):
        info = {"name": value.name, "information": value.information, "source": value.source}
        infos.append(info)
infos_str = ""
for info in infos:
    infos_str += f"\nInformation of '{info['name']}':\n"
    for item in info['information']:
        infos_str += f"  - {item}\n"
    infos_str += f"Source of '{info['name']}':\n"
    for item in info['source']:
        infos_str += f"  - {item}\n"
print(infos_str)



acetonitrile 类在属性 is_coencapsulated_with 上的限制：
类型: SOME
值: classes.guest_molecule(23a)
---
类型: SOME
值: classes.guest_molecule(29)
---
类型: SOME
值: classes.guest_molecule(5a)
---
类型: ONLY
值: classes.acid
---
类型: ONLY
值: classes.guest_molecule(23a) | classes.guest_molecule(29) | classes.guest_molecule(5a)
---

Information of 'guest_molecule(23a)':
  - Another guest molecule encapsulated by the coordination complex.
Source of 'guest_molecule(23a)':
  - https://doi.org/10.1021/acs.accounts.2c00839

Information of 'guest_molecule(29)':
  - A guest molecule that forms a complex with [28a·Pd]2+.
Source of 'guest_molecule(29)':
  - https://doi.org/10.1021/acs.accounts.2c00839

Information of 'guest_molecule(5a)':
  - A guest molecule that is encapsulated by the coordination complex.
Source of 'guest_molecule(5a)':
  - https://doi.org/10.1021/acs.accounts.2c00839

Information of 'acid':
  - A category of substances including hydrochloric acid, hydrobromic acid, and sulfuric acid, used in macroc

# 研究缝隙与方案

## 近邻领域本体
1. 证据缝隙：类的某个值限制具有多个来源，是否存在证据缝隙
2. 知识缝隙：不一致的类
## 多领域本体
1. 方法论缝隙：不同的方法论带来证据的差异
2. 新路径构建（？）
3. 元科学差异：不同领域是否具有深层的“偏见”，能否通过本体结构化



In [15]:
list(acetonitrile.is_a)

[owl.Thing,
 object_properties.is_coencapsulated_with.some(classes.guest_molecule(23a)),
 object_properties.is_coencapsulated_with.some(classes.guest_molecule(29)),
 object_properties.is_coencapsulated_with.some(classes.guest_molecule(5a)),
 object_properties.is_hydrogen_bonded_to.some(classes.pyridyl_α-CHs),
 object_properties.is_coencapsulated_with.only(classes.acid),
 data_properties.reaction_yield_factor_23a.value('1 equiv of 23a'),
 data_properties.reaction_yield_factor_acetonitrile.value('10%'),
 object_properties.acts_as.only(owl.Nothing),
 object_properties.acts_as_stimuli_in.only(owl.Nothing),
 object_properties.binds.only(owl.Nothing),
 object_properties.binds_anion.only(owl.Nothing),
 object_properties.binds_neutral_polar_molecule.only(owl.Nothing),
 object_properties.binds_selectively.only(owl.Nothing),
 object_properties.binds_to.only(owl.Nothing),
 object_properties.binds_with.only(owl.Nothing),
 object_properties.binds_within.only(owl.Nothing),
 object_properties.can_emp

In [14]:
if Nothing in acetonitrile.equivalent_to:
    print("Yes")




Yes


In [7]:
# 遍历每个类并单独进行封闭世界操作
for cls in onto.classes():
    print(cls)
    try:
        close_world(cls)
    except Exception as e:
        print(f"Error closing world for class {cls}: {e}")

classes.oligopyrrolic_cage
classes.pyrrole_subunit
classes.anion_binding
classes.ion_channel_formation
classes.gas_absorption
classes.Cl−⊂3a−f_inclusion_complex
classes.halogen_bonding
classes.anion_supramolecular_research
classes.hydrogen_bonding
classes.catalysis
classes.ion_extraction
classes.responsive_material
classes.anion_p_bonding
classes.anion_supramolecular_chemistry
classes.anion
classes.covalent_oligopyrrolic_macrobicyclic_cage
classes.oligopyrrolic_metallacage
classes.pyrrolic_cage
classes.carbon_dioxide(CO2)
classes.oligopyrrolic_noncovalently_linked_cage
classes.oligopyrrolic_macrocycle
classes.ae-c(4)p
classes.cyclopeptide
classes.β-substituted_oligopyrrole
classes.hybrid_system
classes.heterocycle
classes.pyridine
classes.naphthyridine
classes.oligopyrrolic_polymetallic_cage
classes.oligopyrrolic_supramolecular_cage
classes.dynamic_covalent_chemistry(DCC)
classes.oligopyrrolic_cryptand
classes.cryptand
classes.oligopyrrolic_imine_cage
classes.diformyl_pyrrole_precursor